# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 5
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
train_paths = glob('dataset/cifar/train/*.png')[:100]
test_paths = glob('dataset/cifar/test/*.png')[:100]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

## Training

http://localhost:6006

In [14]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs
)

C:\Users\LGCNS\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
3/3 [==============================] - 6s 1s/step - loss: 2.4160 - accuracy: 0.0303 - val_loss: 2.2846 - val_accuracy: 0.1771
Epoch 2/5
3/3 [==============================] - 0s 170ms/step - loss: 2.3732 - accuracy: 0.0534 - val_loss: 2.3036 - val_accuracy: 0.0833
Epoch 3/5
3/3 [==============================] - 0s 210ms/step - loss: 2.2162 - accuracy: 0.2475 - val_loss: 2.3168 - val_accuracy: 0.0729
Epoch 4/5
3/3 [==============================] - 0s 155ms/step - loss: 2.2689 - accuracy: 0.1132 - val_loss: 2.3400 - val_accuracy: 0.0729
Epoch 5/5
3/3 [==============================] - 0s 184ms/step - loss: 2.1174 - accuracy: 0.3795 - val_loss: 2.3797 - val_accuracy: 0.0729


# Load Checkpoint

In [17]:
checkpoint = model.load_weights('checkpoints')

OpError: 

# Saving Model

In [18]:
save_path = 'my_model.h5'

In [20]:
model.save(save_path, include_optimizer=True)

In [22]:
model = tf.keras.models.load_model('my_model.h5')

# Saving Model - 2

In [23]:
model.save_weights('model_weights.h5')

In [25]:
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())

In [26]:
from tensorflow.keras.models import model_from_json

In [27]:
with open('model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

In [28]:
model.load_weights('model_weights.h5')

# model.h5 들여다보기

In [29]:
import h5py

In [30]:
model_file = h5py.File('my_model.h5', 'r+')

In [35]:
model_file['model_weights'].keys()

<KeysViewHDF5 ['activation', 'activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1', 'dropout', 'dropout_1', 'dropout_2', 'flatten', 'input_1', 'max_pooling2d', 'max_pooling2d_1']>

In [38]:
model_file['model_weights']['conv2d']['conv2d'].keys()

<KeysViewHDF5 ['bias:0', 'kernel:0']>

In [41]:
model_file['model_weights']['conv2d']['conv2d']['kernel:0']

<HDF5 dataset "kernel:0": shape (3, 3, 3, 32), type "<f4">

In [43]:
weight = np.array(model_file['model_weights']['conv2d']['conv2d']['kernel:0'])

In [45]:
weight.shape

(3, 3, 3, 32)

In [46]:
weight

array([[[[ 0.09226803,  0.02035537, -0.07589207,  0.10343052,
          -0.00207525,  0.11616346,  0.04971687, -0.03137767,
          -0.08988912,  0.02444686,  0.09575669, -0.10430526,
           0.12197807,  0.10940919, -0.0496272 ,  0.12526023,
           0.06602866,  0.04394877, -0.09091225,  0.09216809,
           0.04588674, -0.00177437,  0.05969669,  0.11008204,
           0.03956138,  0.10451498, -0.0851523 ,  0.0064488 ,
           0.10007961,  0.13659248, -0.03083892,  0.11937226],
         [-0.14323723,  0.04894153, -0.08499064, -0.12710951,
          -0.00835904,  0.0776788 , -0.08659326, -0.11486722,
           0.07947304, -0.01082734, -0.04861812, -0.08083069,
          -0.08567894,  0.03411101, -0.05327542, -0.00682582,
           0.03834675,  0.0162728 ,  0.0367973 , -0.09071002,
          -0.13866745, -0.08413019, -0.08592813,  0.06848822,
           0.07805333,  0.05277401, -0.02209231, -0.13872363,
           0.05234234,  0.04949731, -0.11033131,  0.11107911],
      